In [1]:
import numpy as np
import cv2 as cv
import os

In [2]:
def show_image(title, image):
    fx = 0.5
    fy = 0.5

    if image.shape[0] > 4000 and image.shape[1] > 3000:
        fx = 0.2
        fy = 0.2

    image = cv.resize(image, (0,0), fx=fx,fy=fy)
    cv.imshow(title, image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
def get_puzzle(img, corners=[]):
    if len(corners) == 0:
        low = np.array([154, 54, 0])
        high= np.array([255, 255, 255])

        img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        mask = cv.inRange(img_hsv, low, high)
        
        kernel = np.ones((3, 3), np.uint8)
        mask = cv.erode(mask, kernel)

        mean = np.mean(mask)
        l = 0.66 * mean
        u = 1.33 * mean
        edges = cv.Canny(mask, l, u)

        contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        top_left = []
        bottom_left = []
        top_right = []
        bottom_right = []

        for i in range(len(contours)):
            if(len(contours[i]) >3):
                possible_top_left = None
                possible_bottom_right = None
                for point in contours[i].squeeze():
                    if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                        possible_top_left = point

                    if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                        possible_bottom_right = point

                diff = np.diff(contours[i].squeeze(), axis = 1)
                possible_top_right = contours[i].squeeze()[np.argmin(diff)]
                possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
                current_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))

                # i want red squares
                # TODO: Use the thid maximum instead of experimental values
                if  current_area >= 15000.0 and current_area <= 20000.0:
                    top_left.append(possible_top_left)
                    bottom_right.append(possible_bottom_right)
                    top_right.append(possible_top_right)
                    bottom_left.append(possible_bottom_left)

        top_left = np.array(top_left)
        top_right = np.array(top_right)
        bottom_left = np.array(bottom_left)
        bottom_right = np.array(bottom_right)

        dist = []
        for tl in top_left:
            dist.append(np.sqrt(tl[0]**2 + tl[1]**2))
        dist = np.array(dist)

        indices = np.argsort(dist)

        top_left = top_left[indices]
        top_right = top_right[indices]
        bottom_left = bottom_left[indices]
        bottom_right = bottom_right[indices]

        c1 = top_left[0]
        c2 = top_right[3]
        c3 = bottom_right[7]
        c4 = bottom_left[4]

        corners = [c1, c2, c3, c4]

        # for tl in top_left:
        #     image_copy = cv.cvtColor(img_hsv, cv.COLOR_HSV2BGR).copy()
        #     cv.circle(image_copy,tuple(tl),10,(0,0,255),-1)
        #     show_image("top_left", image_copy)

        # sum_of_coord_br = []
        # for br in bottom_right:
        #     sum_of_coord_br.append(sum(br))
        # sum_of_coord_br = np.array(sum_of_coord_br)
            
        # min_index = np.argmin(sum_of_coord_tl)
        # c1 = top_left[min_index]

        # max_index = np.argmax(sum_of_coord_br)
        # c3 = bottom_right[np.argmax(max_index)]
        
        # c2 = np.array([c3[0], c1[1]])
        # c4 = np.array([c1[0], c3[1]])

        # print(c1, c2, c3, c4, sep='\n')

        # image_copy = cv.cvtColor(img_hsv, cv.COLOR_HSV2BGR).copy()
        # cv.circle(image_copy,tuple(c1),10,(0,0,255),-1)
        # cv.circle(image_copy,tuple(c2),10,(0,0,255),-1)
        # cv.circle(image_copy,tuple(c3),10,(0,0,255),-1)
        # cv.circle(image_copy,tuple(c4),10,(0,0,255),-1)
        # show_image('The four corners of puzzle', image_copy)

    height, width = 1500, 1500

    puzzle = np.array(corners, dtype = "float32")
    destination_of_puzzle = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = "float32")

    M = cv.getPerspectiveTransform(puzzle, destination_of_puzzle)
    result = cv.warpPerspective(img, M, (width, height))
    
    return (result, corners)

In [4]:
horizontal_lines = []
for i in range(0, 1501, 100):
    l = []
    l.append((0, i))
    l.append((1500, i))
    horizontal_lines.append(l)

In [5]:
vertical_lines = []
for i in range(0, 1501, 100):
    l = []
    l.append((i, 0))
    l.append((i, 1500))
    vertical_lines.append(l)

In [11]:
img = cv.imread("./antrenare/1_01.jpg")
puzzle, corners = get_puzzle(img)

for line in horizontal_lines:
    cv.line(puzzle, line[0], line[1], (0, 255, 0), 5)
for line in vertical_lines:
    cv.line(puzzle, line[0], line[1], (0, 0, 255), 5)
show_image('lines', puzzle)

In [6]:
def vizualizare_configuratie(result, matrix, lines_horizontal, lines_vertical):
    for i in range(len(lines_horizontal) - 1):
        for j in range(len(lines_vertical) - 1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            if matrix[i][j] != 0: 
                cv.rectangle(result, (y_min, x_min), (y_max, x_max), color=(255, 0, 0), thickness=5)
    return result

In [8]:
def get_puzzle_configuration(puzzle, last_positions, horizontal_lines, vertical_lines):
    positions = ''
    matrix_mean = np.zeros((15, 15))
    matrix_positions = np.zeros((15, 15))
    for i in range(len(horizontal_lines)-1):
        for j in range(len(vertical_lines)-1):
            y_min = vertical_lines[j][0][0] + 10
            y_max = vertical_lines[j + 1][1][0] - 10
            x_min = horizontal_lines[i][0][1] + 10
            x_max = horizontal_lines[i + 1][1][1] - 10

            patch = puzzle[x_min:x_max, y_min:y_max].copy()

            low = np.array([80, 0, 0])
            high= np.array([105, 148, 255])

            patch_mask = cv.inRange(cv.cvtColor(patch, cv.COLOR_BGR2HSV), low, high)
            kernel = np.ones((3, 3), np.uint8)
            patch_mask = cv.erode(patch_mask, kernel) 

            mean = np.mean(patch_mask)
            matrix_mean[i, j] = mean

            if mean > 10:
                position = f'{i+1}{chr(j+65)}'
                if position not in last_positions:
                    positions += position + '\n'
                    last_positions.append(position)
                    matrix_positions[i, j] = 1
            
    return (positions, last_positions, matrix_mean, matrix_positions)

In [49]:
path = './antrenare/'
files=sorted(os.listdir(path))

for file in files:
    if file[-3:]=='jpg':
        i, j = file.split('.')[0].split('_')
        image = cv.imread(path + file)

        if j == '01':
            corners = []
            last_postions = []

        #if i == '2': break
            
        puzzle, corners = get_puzzle(image, corners)
        positions, last_postions, matrix_m, matrix_p = get_puzzle_configuration(puzzle, last_postions, horizontal_lines, vertical_lines)

        result = vizualizare_configuratie(puzzle, matrix_p, horizontal_lines, vertical_lines)
        cv.imwrite(f'./result/{i}_{j}.jpg', result)

        f = open(f'./result/{i}_{j}.txt', 'w')
        f.write(positions)

        f = open(f'./result/{i}_{j}_matrix.txt', 'w')
        for i in range(15):
            for j in range(15):
               if(matrix_m[i, j] != 0):
                f.write(f'{i+1}{chr(j+65)}   {matrix_m[i, j]}\n')

        